# Django ORM - Introduction

## 🔹 What is ORM?
ORM stands for Object-Relational Mapping. It's a programming technique that allows you to interact with a database using objects instead of raw SQL queries.

Django's ORM allows you to:
- Define database tables as Python classes (models.Model)
- Query and manipulate the database using Python (QuerySet)
- Avoid writing raw SQL for most operations

## 🔹 Why Use ORM?

| Feature | Benefit |
|---------|---------|
| Abstraction | No need to write raw SQL |
| Code-first | Define models in Python, auto-generate DB tables |
| Cross-DB Support | Works with PostgreSQL, MySQL, SQLite, etc. |
| Safer Queries | Prevents SQL injection, handles escaping |
| Maintainability | Cleaner, more readable code than SQL strings |

## 🔹 Example

```python
# models.py
from django.db import models

class Author(models.Model):
    name = models.CharField(max_length=100)
    country = models.CharField(max_length=100)

class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.ForeignKey(Author, on_delete=models.CASCADE)
    published_year = models.IntegerField()
```

You can now use Python to query the database:

```python
# Fetch books published after 2020
books = Book.objects.filter(published_year__gt=2020)

# Fetch author's name for a specific book
book = Book.objects.get(id=1)
print(book.author.name)
```

Django ORM will automatically translate this into SQL behind the scenes.

# Model Manager - Detailed Explanation

## 🔹 What is a Manager?
Every model in Django has at least one Manager, which is the interface between your model and the database. The default is `objects`.

```python
Book.objects.all()  # 'objects' is the default Manager
```

## 🔹 Purpose of a Manager
- It is the entry point for all database queries
- It returns QuerySets
- You can define custom methods to encapsulate common queries

## 🔹 Built-in Manager: objects

```python
Book.objects.filter(published_year__gte=2020)
Book.objects.get(id=1)
Book.objects.create(title="New Book", published_year=2024)
```

## 🔹 Custom Manager - When and Why?
If your model has repetitive query logic, you can define a custom manager to keep your code DRY and clean.

Example:

```python
from django.db import models

class BookManager(models.Manager):
    def recent(self):
        return self.filter(published_year__gte=2020)

    def by_author(self, author_name):
        return self.filter(author__name=author_name)

class Book(models.Model):
    title = models.CharField(max_length=100)
    published_year = models.IntegerField()
    author = models.ForeignKey('Author', on_delete=models.CASCADE)

    objects = BookManager()
```

Usage:

```python
Book.objects.recent()
Book.objects.by_author("John Doe")
```

This makes your view logic cleaner and pushes query logic into a reusable class.

## 🔹 Manager vs QuerySet

| Feature | Manager | QuerySet |
|---------|---------|----------|
| Entry point | Book.objects | Result of a query: .filter() etc |
| Returns | QuerySet, or object | QuerySet |
| Extensible | Can define custom methods | Yes (via .as_manager()) |

You can also define a custom QuerySet and attach it as a manager:

```python
class BookQuerySet(models.QuerySet):
    def recent(self):
        return self.filter(published_year__gte=2020)

class Book(models.Model):
    title = models.CharField(max_length=100)
    published_year = models.IntegerField()

    objects = BookQuerySet.as_manager()
```

Now, you can chain custom methods:

```python
Book.objects.recent().filter(title__icontains="django")
```

## Summary

| Concept | Description |
|---------|-------------|
| ORM | Lets you interact with the DB using Python classes instead of SQL |
| Model | Represents a table; each attribute = a column |
| Manager | Interface for database operations (objects) |
| Custom Manager | Lets you add custom, reusable query methods |
| QuerySet | A lazy collection of results from the DB |

# QuerySet Basics

## 1. What is a QuerySet?
A QuerySet is a collection of database queries that represents a list of objects retrieved from your database. QuerySets in Django ORM are:
- Lazy: They don't hit the database until actually evaluated.
- Chainable: You can build complex queries step-by-step.
- Iterable: They behave like Python lists in many ways.

## 2. How to Get a QuerySet
You get a QuerySet by calling the model's manager, usually named `objects`.

```python
from myapp.models import Book

qs = Book.objects.all()
```

## 3. Common QuerySet Methods (Play Around)

### all()
Returns all objects from the database.

```python
Book.objects.all()
```

### filter(**kwargs)
Returns a new QuerySet containing objects that match the given lookup parameters.

```python
Book.objects.filter(published_year=2023)
Book.objects.filter(title__icontains='django')  # Case-insensitive partial match
```

### exclude(**kwargs)
Returns a QuerySet excluding the objects matching the given parameters.

```python
Book.objects.exclude(published_year=2023)
```

### order_by()
Sorts the QuerySet by the given fields.

```python
Book.objects.order_by('title')          # Ascending
Book.objects.order_by('-published_year')  # Descending
```

### values() and values_list()
Returns dictionaries or tuples of selected field values instead of full model instances.

```python
Book.objects.values('title', 'published_year')
Book.objects.values_list('title', flat=True)
```

### distinct()
Removes duplicate results.

```python
Book.objects.values('author').distinct()
```

### count()
Returns the number of records matching the query.

```python
Book.objects.filter(published_year=2024).count()
```

### first() and last()
Returns the first or last object in the QuerySet, or None if empty.

```python
Book.objects.order_by('title').first()
Book.objects.order_by('title').last()
```

### exists()
Checks if any records match the query. Returns True or False.

```python
Book.objects.filter(title="Django").exists()
```

### get()
Returns exactly one object that matches the query. Raises DoesNotExist or MultipleObjectsReturned exceptions if not found or too many matches.

```python
Book.objects.get(id=1)
```

### reverse()
Reverses the ordering of the QuerySet.

```python
Book.objects.all().order_by('title').reverse()
```

## Chaining Queries
You can chain multiple QuerySet methods together to refine your query.

```python
Book.objects.filter(title__icontains="django").order_by('-published_year').distinct()
```

## Slicing and Indexing
QuerySets support Python slicing for limiting and paginating results.

```python
Book.objects.all()[:5]     # First 5 records
Book.objects.all()[5:10]   # Next 5 records
Book.objects.all()[0]      # First object
```

## 4. QuerySet Evaluation
QuerySets are lazy and are not executed until you evaluate them. Evaluation happens when:
- You iterate over them
- You call list(), len(), bool(), first(), last(), etc.

```python
qs = Book.objects.filter(published_year=2024)  # No query executed yet
list(qs)  # Triggers the actual query
```

## 5. Summary of Common QuerySet Methods

| Method | Description |
|--------|-------------|
| all() | Returns all objects |
| filter() | Filters objects based on conditions |
| exclude() | Excludes objects matching conditions |
| order_by() | Orders results by given field(s) |
| get() | Gets one object, or raises error |
| first() / last() | Gets first or last object |
| values() | Returns dicts of selected fields |
| values_list() | Returns tuples or flat lists of fields |
| count() | Returns number of records |
| exists() | Checks if any records exist |
| distinct() | Removes duplicates |
| reverse() | Reverses ordering |

## Lazy vs Eager (for comparison)

| Operation | Lazy or Eager | Notes |
|-----------|--------------|-------|
| Book.objects.all() | Lazy | Query not executed |
| list(Book.objects.all()) | Eager | Query executed immediately |
| Book.objects.all().filter(title__icontains='django') | Lazy | Chained, but still not executed |
| Book.objects.all()[0] | Eager | Query executed for first record |

# Advanced QuerySet Methods

## 🔹 get()
Fetch a single object matching the query; raises DoesNotExist if none, or MultipleObjectsReturned if many.

```python
book = Book.objects.get(id=1)
```

## 🔹 create()
Creates and saves a new object in one step.

```python
Book.objects.create(title="New Book", published_year=2023)
```

## 🔹 get_or_create()
Gets the object if it exists, otherwise creates it.

```python
book, created = Book.objects.get_or_create(title="Python 101", defaults={'published_year': 2020})
```

## 🔹 update_or_create()
Gets and updates the object if it exists, or creates it.

```python
book, created = Book.objects.update_or_create(
    title="Python 101", defaults={'published_year': 2024}
)
```

## 🔹 bulk_create()
Creates many objects in a single query (efficient).

```python
Book.objects.bulk_create([
    Book(title="A", published_year=2019),
    Book(title="B", published_year=2020)
])
```

## 🔹 bulk_update()
Efficiently updates multiple objects.

```python
books = Book.objects.all()
for book in books:
    book.published_year += 1
Book.objects.bulk_update(books, ['published_year'])
```

## 🔹 count()
Returns number of records.

```python
Book.objects.filter(published_year=2020).count()
```

## 🔹 in_bulk()
Returns a dictionary of objects keyed by their IDs.

```python
Book.objects.in_bulk([1, 2, 3])
```

## 🔹 iterator()
Efficiently iterates large querysets without loading all into memory.

```python
for book in Book.objects.iterator():
    print(book.title)
```

## 🔹 latest() / earliest()
Get object with latest or earliest value based on a field.

```python
Book.objects.latest('published_year')
Book.objects.earliest('published_year')
```

## 🔹 first() / last()
Returns the first or last object in a queryset.

```python
Book.objects.order_by('title').first()
Book.objects.order_by('title').last()
```

## 🔹 aggregate()
Performs aggregation like Sum, Avg, Max, Min, Count.

```python
from django.db.models import Avg
Book.objects.aggregate(Avg('published_year'))
```

## 🔹 exists()
Checks if a queryset has any rows.

```python
Book.objects.filter(title="Django").exists()
```

## 🔹 update()
Bulk updates rows (doesn't call save()).

```python
Book.objects.filter(title="Old Title").update(title="New Title")
```

## 🔹 delete()
Deletes all objects in queryset.

```python
Book.objects.filter(published_year=2010).delete()
```

## 🔹 as_manager()
Allows using a custom QuerySet as a model manager.

```python
class BookQuerySet(models.QuerySet):
    def recent(self):
        return self.filter(published_year__gte=2020)

class Book(models.Model):
    title = models.CharField(max_length=100)
    published_year = models.IntegerField()

    objects = BookQuerySet.as_manager()
```

## 🔹 explain()
Returns the SQL query execution plan (useful for optimization).

```python
Book.objects.filter(published_year=2020).explain()
```

In [ ]:
class QuerySetDemoView(APIView):
    def get(self, request):
        result = {}

        # create()
        author = Author.objects.create(name="George Orwell")
        book = Book.objects.create(title="1984", published_year=1949, author=author)
        result['create'] = BookSerializer(book).data

        # get_or_create()
        author, created = Author.objects.get_or_create(name="Aldous Huxley")
        result['get_or_create'] = AuthorSerializer(author).data

        # update_or_create()
        book, created = Book.objects.update_or_create(
            title="Brave New World",
            defaults={"published_year": 1932, "author": author}
        )
        result['update_or_create'] = BookSerializer(book).data

        # bulk_create()
        books = [
            Book(title="Book A", published_year=2001, author=author),
            Book(title="Book B", published_year=2002, author=author),
        ]
        Book.objects.bulk_create(books)
        result['bulk_create'] = '2 books created'

        # bulk_update()
        books = Book.objects.filter(author=author)
        for b in books:
            b.published_year += 1
        Book.objects.bulk_update(books, ['published_year'])
        result['bulk_update'] = 'published_year incremented'

        # count()
        result['count'] = Book.objects.count()

        # in_bulk()
        books_dict = Book.objects.in_bulk([1, 2])
        result['in_bulk'] = list(books_dict.keys())

        # latest() and earliest()
        result['latest'] = Book.objects.latest('published_year').title
        result['earliest'] = Book.objects.earliest('published_year').title

        # first() and last()
        result['first'] = Book.objects.order_by('title').first().title
        result['last'] = Book.objects.order_by('title').last().title

        # aggregate()
        result['average_year'] = Book.objects.aggregate(Avg('published_year'))['published_year__avg']

        # exists()
        result['exists'] = Book.objects.filter(title="1984").exists()

        # update()
        Book.objects.filter(title="Book A").update(title="Book A - Updated")
        result['update'] = 'Book A updated'

        # delete()
        Book.objects.filter(title="Book B").delete()
        result['delete'] = 'Book B deleted'

        # as_manager()
        recent_books = Book.objects.recent()
        result['recent_books'] = BookSerializer(recent_books, many=True).data

        # explain()
        explanation = Book.objects.filter(published_year__gte=2000).explain()
        result['explain'] = explanation

        return Response(result)

# Django Relationships

## 1. One-to-One Relationship
Example in your code:

```python
class Profile(models.Model):
    author = models.OneToOneField(Author, on_delete=models.CASCADE)
    biography = models.TextField()
```

Explanation:
A OneToOneField ensures that each Author has only one Profile, and vice versa. It is like extending the Author model with additional information.

Use Case:
Useful when you want to separate optional or sensitive data into another table, like user profile info, settings, or authentication tokens.

## 2. One-to-Many Relationship
Example in your code:

```python
class Book(models.Model):
    author = models.ForeignKey(Author, on_delete=models.CASCADE)
```

Explanation:
A ForeignKey sets up a one-to-many relationship. This means:
- One Author can have many Book records.
- Each Book is linked to only one Author.

Use Case:
Ideal for parent-child relationships like a blog post and comments, or author and books.

## 3. Many-to-Many Relationship
Example in your code:

```python
class Store(models.Model):
    books = models.ManyToManyField(Book)
```

Explanation:
A ManyToManyField allows:
- One Store to have many Book instances.
- One Book to appear in many Store entries.

Use Case:
Use when you need a bidirectional and flexible relationship such as tags on posts, students enrolled in courses, or books in multiple stores.

# Django ORM Cheat Sheet

## 🔹 QuerySet Methods

| Method | One-Liner Description |
|--------|-----------------------|
| filter() | Filters rows matching given conditions (AND logic by default). |
| exclude() | Excludes rows matching given conditions. |
| annotate() | Adds calculated fields (aggregates or expressions) per object. |
| order_by() | Sorts the queryset by one or more fields. |
| reverse() | Reverses the current ordering of the queryset. |
| distinct() | Removes duplicate rows from the queryset. |
| values() | Returns dictionaries of field values (not model instances). |
| values_list() | Returns tuples or flat lists of field values. |
| none() | Returns an empty queryset. |
| all() | Returns all objects in the queryset (no filtering). |
| union() | Combines two querysets and removes duplicates. |
| intersection() | Returns only rows present in both querysets. |
| difference() | Returns rows from the first queryset that are not in the second. |
| select_related() | Performs SQL JOIN to fetch related foreign key data in one query (forward relation). |
| prefetch_related() | Performs two queries and joins them in Python (useful for reverse or M2M). |
| defer() | Defers loading of selected fields until they are accessed. |
| only() | Loads only selected fields immediately; others deferred. |
| using() | Runs the query using a specific database connection alias. |
| select_for_update() | Locks rows during transaction to prevent concurrent updates. |
| raw() | Executes raw SQL queries and returns model instances. |

## 🔹 Expressions

| Expression | One-Liner Description |
|------------|-----------------------|
| F() | Refers to model field values directly in queries for comparison or updates. |
| Q() | Builds complex queries using OR, AND, and NOT logic. |
| Func() | Represents a raw SQL function (e.g., Lower, Length). |
| Value() | Wraps constant values for use in expressions. |
| Aggregate() | Base class for aggregate operations (not used directly). |
| ExpressionWrapper() | Wraps expressions (e.g., arithmetic on F() values) for annotations or filters. |
| SubQuery() | Embeds a subquery in a filter, annotation, or expression. |

## 🔹 Aggregate Functions

| Function | One-Liner Description |
|----------|-----------------------|
| Avg | Calculates average of a field. |
| Count | Counts rows or grouped records. |
| Max | Returns the maximum value of a field. |
| Min | Returns the minimum value of a field. |
| StdDev | Calculates standard deviation of a numeric field. |
| Sum | Calculates total sum of a field. |
| Variance | Calculates statistical variance of a field. |

## 🔹 Field Lookups

| Lookup | One-Liner Description |
|--------|-----------------------|
| contains | Case-sensitive substring match. |
| icontains | Case-insensitive substring match. |
| date | Extracts and compares only the date part of DateTime. |
| day | Matches day part of a date. |
| endswith | Case-sensitive string suffix match. |
| iendswith | Case-insensitive string suffix match. |
| exact | Case-sensitive exact match. |
| iexact | Case-insensitive exact match. |
| in | Matches values in a list or iterable. |
| isnull | Matches NULL or non-NULL values. |
| gt | Greater than comparison. |
| gte | Greater than or equal comparison. |
| hour | Matches hour part of a time or datetime. |
| lt | Less than comparison. |
| lte | Less than or equal comparison. |
| minute | Matches minute part of a time or datetime. |
| month | Matches month part of a date. |
| quarter | Matches quarter of the year (1 to 4). |
| range | Matches if value falls within given range. |
| regex | Case-sensitive regular expression match. |
| iregex | Case-insensitive regular expression match. |
| second | Matches seconds part of a time or datetime. |
| startswith | Case-sensitive string prefix match. |
| istartswith | Case-insensitive string prefix match. |
| time | Matches time part of a DateTime field. |
| week | Matches ISO calendar week number. |
| week_day | Matches day of week (1=Sunday, 7=Saturday). |
| iso_week_day | Matches ISO day of week (1=Monday, 7=Sunday). |
| year | Matches year part of a date or datetime. |
| iso_year | Matches ISO calendar year. |

# ISO Calendar Explanation

## What is the ISO Calendar?
The ISO calendar refers to the ISO 8601 international standard for representing dates and times, maintained by the International Organization for Standardization (ISO). It standardizes how dates, weeks, and times are represented.

## 🔹 Key Concepts of the ISO Calendar (ISO 8601)

| Concept | Description |
|---------|-------------|
| Week starts on | Monday (not Sunday) |
| Week number | Weeks are numbered from 1 to 52/53 |
| First week of year | The week with the first Thursday of the year (or the one containing January 4th) |
| ISO year | May differ from the calendar year if the week spans across two years |

## 🔹 Why it matters in Django
When using Django's ORM field lookups like `iso_year`, `iso_week_day`, and `week`, Django follows the ISO calendar rules.

## 🔹 Example (ISO week vs calendar week)
Consider the date:
January 1, 2023 (Sunday)

- `year`: 2023 — calendar year
- `week_day`: 1 — Sunday (Django default, 1=Sunday)
- `iso_week_day`: 7 — Sunday (ISO: 1=Monday, 7=Sunday)
- `iso_year`: 2022 — because ISO week 1 of 2023 starts on Monday, Jan 2, 2023

## 🔹 Field Lookups in Django (ISO Calendar Related)

| Lookup | Description |
|--------|-------------|
| iso_year | ISO calendar year |
| week | ISO week number (1–53) |
| iso_week_day | ISO day of the week (Monday = 1, Sunday = 7) |

## 🔹 When to Use ISO Calendar?
- When dealing with week-based analytics or calendars (e.g., weekly reports)
- To conform to international standards
- When consistent weekday/week logic is needed regardless of locale

# Django ORM Concepts

## 🔹 Forward vs Reverse Relation in Django (Short Explanation)
- **Forward Relation**: Access from the model that defines the relationship.
  → Example: `student.classroom` (Student has FK to Classroom)
  
- **Reverse Relation**: Access from the related model back to the source.
  → Example: `classroom.students.all()` (Classroom retrieves related Students via related_name or student_set)

## 🔹 aggregate() vs annotate() (Short Explanation)
- **aggregate()**: Computes a summary value for the entire queryset (e.g., total, average).
  → Example: `Student.objects.aggregate(Avg('marks'))`
  
- **annotate()**: Computes and attaches values per object in the queryset.
  → Example: `Classroom.objects.annotate(student_count=Count('students'))`